In [1]:
import os
import random
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Gain

# -------------------------
# Paths & Config
# -------------------------
DATA_DIR = r"E:\Data"
RAVDESS_PATH = os.path.join(DATA_DIR, "ravdess")
TESS_PATH    = os.path.join(DATA_DIR, "tess")
CREMAD_PATH  = os.path.join(DATA_DIR, "cremad")

SR = 22050
DURATION = 3.0
N_MELS = 128
BATCH_SIZE = 32
NUM_EPOCHS = 50  # Increased epochs for better accuracy
LR = 1e-3
SEED = 42
MODEL_SAVE_PATH = "best_emotion_cnn.pth"

os.makedirs("features", exist_ok=True)

# -------------------------
# Reproducibility
# -------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [2]:
augmenter = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, leave_length_unchanged=True, p=0.4),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
    Shift(min_shift=-0.5, max_shift=0.5, rollover=True, p=0.5),
    Gain(min_gain_db=-6, max_gain_db=6, p=0.4),
])

def extract_features(wav_path, sr=SR, n_mels=N_MELS, duration=DURATION, apply_augment=False):
    y, sr = librosa.load(wav_path, sr=sr, duration=duration)
    target_len = int(sr * duration)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]

    if apply_augment:
        y = augmenter(samples=y.astype(np.float32), sample_rate=sr)

    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_mel = librosa.power_to_db(mel, ref=np.max)
    log_mel = (log_mel - np.mean(log_mel)) / (np.std(log_mel) + 1e-9)
    return log_mel.astype(np.float32)


In [3]:
class EmotionDataset(Dataset):
    def __init__(self, paths, labels, augment=False):
        self.paths = paths
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = extract_features(self.paths[idx], apply_augment=self.augment)
        x = torch.tensor(x).unsqueeze(0)
        y = torch.tensor(self.labels[idx]).long()
        return x, y

def gather_audio_files(data_root, exts=(".wav", ".WAV")):
    files = []
    for root, dirs, filenames in os.walk(data_root):
        for f in filenames:
            if f.endswith(exts):
                files.append(os.path.join(root, f))
    return files


In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout=0.3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        out = self.dropout(out)
        return out

class UltraStrongCNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.init_conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.layer1 = ResidualBlock(32, 64, stride=2, dropout=0.2)
        self.layer2 = ResidualBlock(64, 128, stride=2, dropout=0.3)
        self.layer3 = ResidualBlock(128, 256, stride=2, dropout=0.4)
        self.layer4 = ResidualBlock(256, 512, stride=2, dropout=0.5)
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, n_classes)

    def forward(self, x):
        out = self.init_conv(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [5]:
def train_one_epoch(model, loader, optimizer, criterion, device, epoch):
    model.train()
    losses, preds, trues = [], [], []
    pbar = tqdm(loader, desc=f"Train Epoch {epoch}", unit="batch")
    for xb, yb in pbar:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        preds.extend(torch.argmax(out,1).cpu().numpy())
        trues.extend(yb.cpu().numpy())
        pbar.set_postfix(loss=np.mean(losses), acc=(np.array(preds)==np.array(trues)).mean())
    return np.mean(losses), (np.array(preds)==np.array(trues)).mean()

def eval_model(model, loader, criterion, device, epoch):
    model.eval()
    losses, preds, trues = [], [], []
    with torch.no_grad():
        pbar = tqdm(loader, desc=f"Eval Epoch {epoch}", unit="batch")
        for xb, yb in pbar:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            losses.append(loss.item())
            preds.extend(torch.argmax(out,1).cpu().numpy())
            trues.extend(yb.cpu().numpy())
            pbar.set_postfix(loss=np.mean(losses), acc=(np.array(preds)==np.array(trues)).mean())
    return np.mean(losses), (np.array(preds)==np.array(trues)).mean()


In [6]:
def main():
    # Gather files
    file_list = []
    for p in [RAVDESS_PATH, TESS_PATH, CREMAD_PATH]:
        if os.path.isdir(p):
            file_list.extend(gather_audio_files(p))
    print("Total audio files:", len(file_list))

    # Label extraction
    labels, paths = [], []
    for f in file_list:
        base = os.path.basename(f).lower()
        if "ang" in base: labels.append("angry")
        elif "calm" in base: labels.append("calm")
        elif "dis" in base: labels.append("disgust")
        elif "fea" in base: labels.append("fear")
        elif "hap" in base: labels.append("happy")
        elif "neu" in base: labels.append("neutral")
        elif "sad" in base: labels.append("sad")
        elif "sur" in base: labels.append("surprise")
        else: continue
        paths.append(f)

    le = LabelEncoder()
    y = le.fit_transform(labels)
    print("Classes:", list(le.classes_))

    # Split
    X_train, X_val, y_train, y_val = train_test_split(paths, y, test_size=0.2, stratify=y, random_state=SEED)

    # Datasets
    train_ds = EmotionDataset(X_train, y_train, augment=True)
    val_ds = EmotionDataset(X_val, y_val, augment=False)

    # DataLoaders
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=False)
    val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=False)

    # Model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UltraStrongCNN(n_classes=len(le.classes_)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    best_val_acc = 0
    for epoch in range(1, NUM_EPOCHS+1):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)
        val_loss, val_acc = eval_model(model, val_loader, criterion, device, epoch)
        print(f"\nEpoch {epoch}/{NUM_EPOCHS} | Train acc: {train_acc:.4f}, loss: {train_loss:.4f} | Val acc: {val_acc:.4f}, loss: {val_loss:.4f}")
        scheduler.step(val_loss)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({'model_state_dict': model.state_dict(), 'label_encoder': le}, MODEL_SAVE_PATH)
            print(f"Saved new best model with val_acc {best_val_acc:.4f}")

if __name__=="__main__":
    main()


Total audio files: 11682
Classes: [np.str_('angry'), np.str_('calm'), np.str_('disgust'), np.str_('fear'), np.str_('happy'), np.str_('neutral'), np.str_('sad'), np.str_('surprise')]


Eval Epoch 1: 100%|███████████████████████████████████████████| 62/62 [02:07<00:00,  2.06s/batch, acc=0.475, loss=1.32]



Epoch 1/50 | Train acc: 0.4156, loss: 1.4416 | Val acc: 0.4746, loss: 1.3226
Saved new best model with val_acc 0.4746


Eval Epoch 2: 100%|███████████████████████████████████████████| 62/62 [01:24<00:00,  1.36s/batch, acc=0.566, loss=1.13]



Epoch 2/50 | Train acc: 0.5179, loss: 1.2319 | Val acc: 0.5665, loss: 1.1282
Saved new best model with val_acc 0.5665


...


Epoch 48/50 | Train acc: 0.8464, loss: 0.4139 | Val acc: 0.7584, loss: 0.6966


Eval Epoch 49: 100%|█████████████████████████████████████████| 62/62 [01:20<00:00,  1.30s/batch, acc=0.766, loss=0.704]



Epoch 49/50 | Train acc: 0.8464, loss: 0.4074 | Val acc: 0.7665, loss: 0.7036
Saved new best model with val_acc 0.7665


Eval Epoch 50: 100%|█████████████████████████████████████████| 62/62 [01:21<00:00,  1.31s/batch, acc=0.772, loss=0.676]


Epoch 50/50 | Train acc: 0.8606, loss: 0.3808 | Val acc: 0.7721, loss: 0.6755
Saved new best model with val_acc 0.7721
